writing my own code (COPY-PASTING CODE NOT ALLOWED, DOC IS OK) for cs231n's classification notes

this notebook is for: https://cs231n.github.io/optimization-1/

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [3]:
from utils import load_cifar
Xtr_raw, Ytr, Xte_raw, Yte = load_cifar()

In [4]:
def preprocess(X): return (X - X.mean()) / (X.max() - X.min())

In [5]:
# adding biases
def add_bias(X): return np.hstack([X, np.ones((X.shape[0], 1))])
Xtr, Xte = map(add_bias, map(preprocess, [Xtr_raw, Xte_raw]))

In [6]:
labels = list(set(np.unique(Ytr)).union(set(np.unique(Yte))))
m, d = Xtr.shape
ncl = len(labels)

In [7]:
class Classifier:
    def L(self, X, y, W):
        score = np.dot(W, X)
        score_lab = score[y, np.arange(X.shape[1])]
        loss =  np.sum(np.maximum(0, score - score_lab + 1)) - score_lab.shape[0]
        return loss / X.shape[1]
    def RandomSearch(self, X, y, ncl, n_iter=1000):
        best_loss = np.inf
        d = X.shape[0]
        for i in range(n_iter):
            W = np.random.randn(ncl, d) * 1e-4
            loss = self.L(X, y, W)
            if loss < best_loss:
                Wbest = W
                best_loss = loss
            if i > 0 and i % 10 == 0:
                print(f"after {i} iteration, best loss={best_loss}")
        return Wbest
    def LocalSearch(self, X, y, ncl, n_iter=1000):
        best_loss = np.inf
        d = X.shape[0]
        W = np.random.randn(ncl, d) * 1e-4
        alpha = 1e-4
        for i in range(n_iter):
            delta = np.random.randn(ncl, d)
            W_p = W + alpha * delta
            loss = self.L(X, y, W_p)
            if loss < best_loss:
                W = W_p
                best_loss = loss
            if i > 0 and i % 10 == 0:
                print(f"after {i} iteration, best loss={best_loss}")
        return W

In [8]:
def test_accuracy(W):
    Yhat = np.argmax(np.dot(W, Xte.T), axis=0)
    print(f"accuracy: {100 * np.mean(Yte == Yhat)}%")

In [9]:
clf = Classifier()
#Wbest = clf.RandomSearch(Xtr.T, Ytr, ncl, n_iter=1000)
#test_accuracy(Wbest)
#Wbest_local = clf.LocalSearch(Xtr.T, Ytr, ncl, n_iter=1000)
#test_accuracy(Wbest_local)

In [10]:
def eval_numerical_gradient(f, x, h=1e-5):
    fx = f(x)
    grad = np.zeros(x.shape)
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp = x[idx]
        x[idx] += h
        fxh = f(x)
        x[idx] = tmp
        grad[idx] = (fxh - fx) / h
        it.iternext()
    return grad

In [11]:
def cifar_loss(W, n_ex_per_loss=100):
    clf = Classifier()
    return clf.L(Xtr[:n_ex_per_loss].T, Ytr[:n_ex_per_loss], W)

In [13]:
import time
n_ex_per_loss = 1
start = time.time()
W = np.random.randn(ncl, d) * 1e-4
df = eval_numerical_gradient(cifar_loss, W)
print(f"gradient with {n_ex_per_loss} examples per loss took {time.time()-start:2f}s")

gradient with 1 examples per loss took 9.956538s


In [14]:
W = np.random.randn(ncl, d) * 1e-4
loss_0 = cifar_loss(W)
print(f"gradient has been computed with {n_ex_per_loss} examples in the loss")
for i in list(np.arange(-10, 0)):
    alpha = float(10) ** i
    W_new = W - df * alpha
    loss = cifar_loss(W_new)
    print(f"loss for alpha={alpha} is {loss:2f}")

gradient has been computed with 1 examples in the loss
loss for alpha=1e-10 is 8.999640
loss for alpha=1e-09 is 8.999639
loss for alpha=1e-08 is 8.999636
loss for alpha=1e-07 is 8.999606
loss for alpha=1e-06 is 8.999306
loss for alpha=1e-05 is 8.996306
loss for alpha=0.0001 is 8.966308
loss for alpha=0.001 is 8.666319
loss for alpha=0.01 is 6.051703
loss for alpha=0.1 is 5.931562


In [15]:
def svm_gradient(x, y, W):
    d = x.shape[0]
    ncl = W.shape[0]
    score = np.dot(W, x)
    grad = np.zeros_like(W)
    sum_not_lab = np.sum(np.maximum(0, score - score[y] + 1)) - 1
    for j in range(ncl):
        if j != y:
            grad[j] = ((score[j] - score[y] + 1) > 0) * x
        else:
            grad[j] = -sum_not_lab * x
    return grad

In [29]:
def svm_gradient_all(X, y, W):
    dw = np.zeros_like(W)
    for j in range(X.shape[1]):
        dw += svm_gradient(X[:,j], y[j], W)
    return dw

In [30]:
out = svm_gradient(Xtr[0].T, Ytr[0], W)
svm_gradient_all(Xtr[:10].T, Ytr[:10], W).shape

(10, 3073)

In [31]:
def vanilla_gradient_descent(W_init, n_steps=1):
    W = W_init
    alpha = 0.01
    for step in range(n_steps):
        print(f"step #{step}")
        dW = svm_gradient_all(Xtr.T, Ytr, W)
        W -= alpha * dW

In [32]:

vanilla_gradient_descent(W)
test_accuracy(W)

step #0
accuracy: 26.88%


In [20]:
def sample_batch(data, batch_size):
    return data[random.sample(set(np.arange(data.shape[0])), batch_size)]

In [21]:
def minibatch_gradient_descent(W_init, n_steps=1, batch_size=256, alpha=1e-4):
    W = W_init
    for step in range(n_steps):
        X, Y = map(lambda x: sample_batch(x, batch_size), [Xtr, Ytr])
        dW = svm_gradient_all(X.T, Y, W)
        W -= alpha * dW

In [38]:
# pls someone explain why accuracy doesn't go up here
W = np.random.randn(ncl, d) * 1e-4
alpha = 0.01
for batch in range(100):
    minibatch_gradient_descent(W, n_steps=1, batch_size=Xtr.shape[0], alpha=alpha)
    if batch > 0 and batch % 10 == 0:
        print(f"batch #{batch}")
        test_accuracy(W)
        print(f"loss={cifar_loss(W, 100)}")

batch #10
accuracy: 10.639999999999999%
loss=2.7159744433261036e+44
batch #20
accuracy: 10.290000000000001%
loss=6.827552204341103e+85
batch #30
accuracy: 10.47%
loss=1.734563596764155e+127
batch #40
accuracy: 11.44%
loss=4.0502986318315826e+168
batch #50
accuracy: 11.33%
loss=1.0308672365292907e+210
batch #60
accuracy: 10.51%
loss=2.6316739501843706e+251
batch #70
accuracy: 10.5%
loss=6.455395688706882e+292
batch #80
accuracy: 10.0%
loss=nan
batch #90
accuracy: 10.0%
loss=nan
